### Fine-tuning a model with Keras
Once you’ve done all the data preprocessing work in the last section, you have just a few steps left to train the model. Note, however, that the model.fit() command will run very slowly on a CPU. If you don’t have a GPU set up, you can get access to free GPUs or TPUs on Google Colab.

The code examples below assume you have already executed the examples in the previous section. Here is a short summary recapping what you need:

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-feda42d5ca96fd22.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-4689576af2978785.arrow


### Training
TensorFlow models imported from 🤗 Transformers are already Keras models. That means that once we have our data, very little work is required to begin training on it.

As in the previous chapter, we will use the **TFAutoModelForSequenceClassification** class, with two labels:

In [8]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You will notice that unlike in Chapter 2, you get a warning after instantiating this pretrained model. **This is because BERT has not been pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been inserted instead.** The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

**To fine-tune the model on our dataset**, we just have to **compile()** our model and **then pass our data to the fit() method**. This will start the fine-tuning process (which should take a couple of minutes on a GPU) and report training loss as it goes, plus the validation loss at the end of each epoch.

Note that 🤗 Transformers models have a special ability that most Keras models don’t - they can automatically use an appropriate loss which they compute internally. **They will use this loss by default if you don’t set a loss argument in compile().** Note that to use the internal loss you’ll need to pass your labels as part of the input, not as a separate label, which is the normal way to use labels with Keras models. You’ll see examples of this in Part 2 of the course, where defining the correct loss function can be tricky. For sequence classification, however, a standard Keras loss function works fine, so that’s what we’ll use here.

In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

458/458 [==============================] - 83s 152ms/step - loss: 0.6819 - accuracy: 0.6367 - val_loss: 0.6919 - val_accuracy: 0.6838


Note a very common pitfall here — you can just pass the name of the loss as a string to Keras, **but by default Keras will assume that you have already applied a softmax to your outputs. Many models, however, output the values right before the softmax is applied, which are also known as the logits.** We need to tell the loss function that that’s what our model does, and the only way to do that is to call it directly, rather than by name with a string.

If you try the above code, it certainly runs, but you’ll find that the loss declines only slowly or sporadically. The primary cause is the **learning rate**. As with the loss, when we pass Keras the name of an optimizer as a string, Keras initializes that optimizer with default values for all parameters, including learning rate. From long experience, though, we know that **transformer models benefit from a much lower learning rate than the default for Adam, which is 1e-3, also written as 10 to the power of -3, or 0.001. 5e-5 (0.00005), which is some twenty times lower, is a much better starting point.**

In addition to lowering the learning rate, we have a second trick up our sleeve: **We can slowly reduce the learning rate over the course of training.** In the literature, you will sometimes see this referred to as **decaying or annealing the learning rate.** In Keras, the best way to do this is to use a **learning rate scheduler**. A good one to use is **PolynomialDecay** — despite the name, with default settings it simply linearly decays the learning rate from the initial value to the final value over the course of training, which is exactly what we want. In order to use a scheduler correctly, though, we need to tell it how long training is going to be. We compute that as **num_train_steps** below.

In [12]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam

batch_size = 8
num_epochs = 3

# The number of training steps is the number of samples in the dataset, divided by the batch size 
#then multiplied by the total number of epochs
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

The 🤗 Transformers library also has a **create_optimizer()** function that will create an AdamW optimizer with learning rate decay. This is a convenient shortcut that you’ll see in detail in future sections of the course.

Now we have our all-new optimizer, and we can try training with it. **First, let’s reload the model, to reset the changes to the weights from the training run we just did, and then we can compile it with the new optimizer:**

In [13]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we fit again:

In [14]:
model.fit(
    tf_train_dataset, 
    validation_data=tf_validation_dataset,
    epochs=3
    )

Epoch 1/3
458/458 [==============================] - 83s 152ms/step - loss: 0.5632 - accuracy: 0.7039 - val_loss: 0.4223 - val_accuracy: 0.8039
Epoch 2/3
458/458 [==============================] - 68s 147ms/step - loss: 0.3456 - accuracy: 0.8510 - val_loss: 0.3530 - val_accuracy: 0.8309
Epoch 3/3
458/458 [==============================] - 69s 150ms/step - loss: 0.1326 - accuracy: 0.9563 - val_loss: 0.4240 - val_accuracy: 0.8431


💡 If you want to automatically upload your model to the Hub during training, you can pass along a **PushToHubCallback** in the model.fit() method. We will learn more about this in Chapter 4

### Model predictions
Training and watching the loss go down is all very nice, but what if we want to actually get outputs from the trained model, either to compute some metrics, or to use the model in production? To do that, we can just use the **predict()** method. **This will return the logits from the output head of the model, one per class.**

In [17]:
preds = model.predict(tf_validation_dataset)["logits"]

We can convert these logits into the model’s class predictions **by using argmax to find the highest logit, which corresponds to the most likely class:**

In [19]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(408, 2) (408,)


Now, let’s use those preds to compute some metrics! We can load the metrics associated with the MRPC dataset as easily as we loaded the dataset, this time with the **load_metric()** function. The object returned has a **compute()** method we can use to do the metric calculation:

In [20]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

{'accuracy': 0.8431372549019608, 'f1': 0.8900343642611684}

The exact results you get may vary, as the random initialization of the model head might change the metrics it achieved. Here, we can see our model has an accuracy of 84.31% on the validation set and an F1 score of 89.00. Those are the two metrics used to evaluate results on the MRPC dataset for the GLUE benchmark. The table in the BERT paper reported an F1 score of 88.9 for the base model. That was the uncased model while we are currently using the cased model, which explains the better result.

This concludes the introduction to fine-tuning using the Keras API. An example of doing this for most common NLP tasks will be given in Chapter 7.